# Lập trình Python: Giới thiệu về Decorator 

### BS. Lê Ngọc Khả Nhi

## Function là 1 first-class object

Trong bài này, Nhi sẽ giới thiệu với các bạn về Decorator, tính năng cho phép thay đổi cấu trúc hàm trong Python. 

Trước khi nói về Decorator, ta nhắc lại vài khái niệm cơ bản về hàm (function). Một cách đơn giản, hàm là một cấu trúc hoạt động theo cơ chế: tiếp nhận tham số đầu vào, thực thi một số công việc và xuất kết quả ở đầu ra, thí dụ hàm tính BMI nhận giá trị Height, Weight như arguments, tính và xuát kết quả BMI:

In [2]:
def BMI(h,w):
    bmi = w/(0.01*h)**2
    return bmi

In [3]:
BMI(173, 54)

18.042701059173375

Trong Python, mọi thứ đều là object, nhưng hàm đặc biệt hơn vì nó là 1 "object hạng nhất", nó được phép đi vào mọi nơi trong chương trình, thí dụ ta có thể dùng hàm này như argument của hàm khác, một hàm lớn có thể chứa các hàm nhỏ hơn bên trong, và xuất kết quả là một hàm khác nữa.

Trong thí dụ sau, hàm BMI iter chứa 1 hàm BMI nhỏ hơn bên trong và sử dụng hàm này để tính BMI rồi so sánh với ngưỡng 30 và xuất kết quả:

In [28]:
def BMI_inter(h,w):
    def BMI(h,w):
        bmi = w/(0.01*h)**2
        return bmi
    out = BMI(h,w)
    return f'BMI = {out:.2f}: Thuộc loại béo phì' if out > 30.0 else f'BMI = {out: .3f}'

In [29]:
BMI_inter(h=165, w=68)

'BMI =  24.977'

In [30]:
BMI_inter(h=165, w=82)

'BMI = 30.12: Thuộc loại béo phì'

Một phiên bản khác của hàm BMI_iter nhận function lambda làm argument, và xuất ra 1 hàm và 1 kết quả

In [36]:
def BMI_inter2(h,w, func= lambda h,w: w/(0.01*h)**2):
    out = func(h,w)
    return func, f'BMI = {out:.2f}: Thuộc loại béo phì' if out > 30.0 else f'BMI = {out: .3f}'

In [39]:
bmi_func, res = BMI_inter2(h=165, w=82)

print(res)

print(bmi_func(165,82))

BMI = 30.12: Thuộc loại béo phì
30.119375573921022


## Decorator đơn giản: không thay đổi tính năng hàm 

Như vậy, bạn có thể thấy rằng function có thể được dùng với vai trò rất linh hoạt.

Bây giờ ta nói về decorator; về bản chất thì decorator cũng là 1 hàm (function), nhưng nó nhận argument là 1 function, và có khả năng đóng gói, thi hành, thay đổi cấu trúc và tính năng của hàm này.

Trong 1 thí dụ đơn giản nhất, ta muốn kiểm tra khi nào 1 hàm my_func được thi hành, và khi nào nó kết thúc; ta sẽ tạo một decorator tên là test:

1) test nhận my_func làm argument

2) bên trong test có 1 hàm exec_func() làm nhiệm vụ: in 2 dòng thông báo khi hàm my_func bắt đầu và kết thúc

3) Hàm exec_func đóng gói hàm my_func và xuất ra dưới 1 object là out

4) test xuất ra kết quả là exec_func()

Khi chưa dùng decorator test, hàm my_func không có tính năng thông báo khi nào nó chạy hay kết thúc:

In [41]:
import time

def my_func():
    print('Bạn đang chạy hàm my_func')
    time.sleep(3)
    return 'Đây là kết quả của hàm my_func'

my_func()

Bạn đang chạy hàm my_func


'Đây là kết quả của hàm my_func'

In [43]:
import time

def test(function):
    def exec_func():
        print(f'Bắt đầu thi hành hàm {function}')
        out = function()
        print(f'Kết thúc hàm {function}')
        return out
    return exec_func

Decorator được dùng bằng cách đặt tên của nó sau kí tự @, và ngay trước đoạn code khai báo hàm my_func

In [44]:
@test
def my_func():
    print('Bạn đang chạy hàm my_func')
    time.sleep(3)
    return 'Đây là kết quả của hàm my_func'

Lúc này hàm my_func có thêm tính năng thông báo bắt đầu và kết thúc mà bản thân nó không có trước kia

In [45]:
my_func()

Bắt đầu thi hành hàm <function my_func at 0x00000173E719A048>
Bạn đang chạy hàm my_func
Kết thúc hàm <function my_func at 0x00000173E719A048>


'Đây là kết quả của hàm my_func'

## Decorator làm thay đổi tính năng hàm mục tiêu

Bây giờ, ta sẽ decorator để thay đổi tính năng của 1 hàm mục tiêu. Trong thí dụ sau: Ta có hàm read_exam() làm nhiệm vụ so sánh kết quả 1 xét nghiệm res và giá trị tham chiếu normal, nếu tỉ lệ res/normal thấp hơn 80%, in kết quả bất thường

In [55]:
def read_exam(res = None, normal = 100):
    out = res/normal
    inter = 'Bất thường' if out < 0.8 else 'Bình thường'
    print(f"Kết quả xét nghiệm = {out: .3f} % giá trị tham chiếu:", inter)

read_exam(res = 79, normal = 100)

Kết quả xét nghiệm =  0.790 % giá trị tham chiếu: Bất thường


Giả sử ta muốn điều chỉnh một cách hệ thống giá trị tham chiếu cho chủng tộc châu Á, theo quy luật, người châu Á có giá trị normal thấp hơn 10% so với người Âu, ta có thể dùng 1 decorator asian như sau:

1) Decorator asian nhận function read_exam() như argument

2) Bên trong asian, ta tạo 1 function inner_func có cấu trúc giống như read_exam, nhưng tính lại normal = 90% của normal

3) Thi hành hàm func bên trong hàm inner_func() và xuất kết quả

3) asian xuất kết quả là hàm inner_func

In [57]:
def asian(func):
    def inner_func(res, normal):
        normal *= 0.9
        return func(res, normal)
    return inner_func

In [58]:
@asian
def read_exam(res = None, normal = 100):
    out = res/normal
    inter = 'Bất thường' if out < 0.8 else 'Bình thường'
    print(f"Kết quả xét nghiệm = {out: .3f} % giá trị tham chiếu:", inter)

Sau khi dùng decorator asian, ta có 1 hàm read_exam mới, trong đó argument normal được tự động trừ đi 10%

In [59]:
read_exam(res = 78, normal = 100)

Kết quả xét nghiệm =  0.867 % giá trị tham chiếu: Bình thường


## Dùng decorator để đo thời gian thi hành function

Một ứng dụng phổ biến của decorator là để đo hiệu suất của hàm, ta muốn biết nó thi hành mất bao lâu:

Trong thí dụ sau, ta tạo ra decorator timer và áp dụng cho hàm my_func.

Bên trong decorator timer, ta đóng gói hàm my_func vào 1 hàm inner_func, để đưa thêm tính năng đo đếm thời gian:

In [76]:
def timer(func):
    def inner_func(*args):
        start = time.time()
        out = func(*args)
        duration = time.time() - start
        print(f'Hàm {func} chạy mất {duration} giây')
        return out
    return inner_func

In [77]:
@timer
def my_func(dt):
    print('Hàm my_func đang chạy')
    time.sleep(dt)
    return 'Hàm my_func chạy xong'

In [78]:
my_func(8)

Hàm my_func đang chạy
Hàm <function my_func at 0x00000173E719A048> chạy mất 8.013370990753174 giây


'Hàm my_func chạy xong'

## Decorator có chứa argument

Trong các thí dụ trên, decorator là một cấu trúc bất biến, tuy nhiên ta có thể tạo decorator có chứa tham số (argument) và dùng argument này để thay đổi tính năng decorator; 

Trong thí dụ sau, ta cùng làm lại decorator asian, nhưng có thêm argument 'coef' cho phép xác định % hiệu chỉnh của normal, thí dụ 12% hay 15% thay vì 10% cố định:

In [65]:
def asian(coef):
    def adjuster(func):
        def new_func(**kwargs):
            res = kwargs['res']
            normal = kwargs['normal']
            normal *= (1-coef)
            return func(res, normal)
        return new_func
    return adjuster

In [66]:
@asian(0.15)
def read_exam(res = 78, normal = 100):
    print(f'Kết quả = {res/normal} % giá trị tham chiếu')

In [67]:
read_exam(res = 78, normal = 100)

Kết quả = 0.9176470588235294 % giá trị tham chiếu


In [68]:
@asian(0.13)
def read_exam(res = 78, normal = 100):
    print(f'Kết quả = {res/normal} % giá trị tham chiếu')

In [69]:
read_exam(res = 78, normal = 100)

Kết quả = 0.896551724137931 % giá trị tham chiếu


## Dùng decorator để kiểm tra dữ liệu đầu vào

Trong 1 thí dụ khác, ta có thể dùng decorator để kiểm tra dữ liệu argument của 1 hàm có phải là float, integer hay string hay không ?

Thí dụ: decorator check_float kiểm tra argument res của hàm read_exam có phải là float number, nếu không, sẽ báo lỗi, nếu có sẽ tiếp tục thi hành 

In [70]:
def check_float(func):
    def new_func(**kwargs):
        res = kwargs['res']
        if not isinstance(res, float):
            raise Exception('res phải là float number')
        return func(**kwargs)
    return new_func

In [71]:
@check_float
def read_exam(res = 78, normal = 100):
    print(f'Kết quả = {res}')

In [72]:
read_exam(res = 78, normal = 100)

Exception: res phải là float number

In [74]:
read_exam(res = 78.0, normal = 100)

Kết quả = 78.0 là float number


## Decorator kép
    
Có thể déung đồng thời nhiều decorator, như trong thí dụ đơn giản sau, ta dùng 2 decorator: print_flag thông báo hàm bắt đầu thi hành, và timer để đo thời gian:

In [79]:
def print_flag(func):
    def new_function(dt):
        print('Bắt đầu')
        return func(dt)
    return new_function

def timer(func):
    def new_function(dt):
        start = time.time()
        out = func(dt)
        end = time.time()
        dur = end - start
        print(f'Hàm chạy mất {dur} giây')
        return out
    return new_function

In [80]:
@print_flag
@timer
def my_func(dt = 7):
    time.sleep(dt)
    return 'Kết quả hàm my_func'

In [81]:
my_func(dt = 7)

Bắt đầu
Hàm chạy mất 7.003093481063843 giây


'Kết quả hàm my_func'

## Decorator làm thay đổi kết quả hàm mục tiêu

Ở trên, ta thấy có thể dùng decorator để điều chỉnh argument của hàm mục tiêu trước khi thi hành nó; trong thí dụ tiếp theo này, ta không thay đổi argument nhưng thay đổi trực tiếp kết quả của hàm mục tiêu ngay sau khi nó được thi hành;

Ta dùng decorator inches để đổi giá trị chiều cao của hàm height từ cm sang inch:

In [82]:
def inches(func):
    def height_entry(**kwargs):
        height = kwargs['height']
        out = func(**kwargs)
        return f'Chiều cao = {out*0.393701} inches'
    return height_entry


def height_input(height):
    print(f'Chiều cao = {height} cm')
    return height

In [83]:
height_input(height = 176)

Chiều cao = 176 cm


176

In [84]:
@inches
def height_input(height):
    print(f'Chiều cao = {height} cm')
    return height

In [85]:
height_input(height = 176)

Chiều cao = 176 cm


'Chiều cao = 69.291376 inches'

## Hàm mục tiêu không mất đi đâu cả...

Sau khi áp dụng decorator, hàm mục tiêu đã bị thay đổi tính năng và không còn là chính nó nữa, tuy nhiên ta có thể lấy lại phiên bản nguyên thủy của hàm này:

Quay trở lại decorator asian và hàm read_exam

In [89]:
def asian(func):
    def inner_func(res, normal):
        normal *= 0.9
        return func(res, normal)
    return inner_func

@asian
def read_exam(res = 78, normal = 100):
    print(f'Kết quả = {res/normal} % giá trị tham chiếu')

Sau khi dùng decorator, hàm read_exam là 1 objet có vị trí sau:

In [90]:
read_exam

<function __main__.asian.<locals>.inner_func(res, normal)>

Nếu ta truy nhập vào sâu bên trong object này, có thể lấy lại hàm read_exam gốc tại : read_exam.\__closure\__[0].cell_contents

In [91]:
read_exam.__closure__[0]

<cell at 0x00000173E71851C8: function object at 0x00000173E7186A68>

In [92]:
read_exam.__closure__[0].cell_contents

<function __main__.read_exam(res=78, normal=100)>

In [93]:
origin_func = read_exam.__closure__[0].cell_contents

Hàm origin_function là phiên bản gốc của hàm read_exam trước khi bị thay đổi:

In [94]:
read_exam(res = 78, normal = 100)

Kết quả = 0.8666666666666667 % giá trị tham chiếu


In [95]:
origin_func(res = 78, normal = 100)

Kết quả = 0.78 % giá trị tham chiếu


Bài thực hành về decorator tạm dừng ở đây, chúc các bạn thực hành vui và hẹn gặp lại.